In [ ]:
import time
import random
import pandas as pd
import numpy as np
from pathlib import Path
import os.path
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_pickle('data/2020_stats_final.pkl')
df.columns = [column.capitalize() for column in df.columns]
df

In [ ]:
def extract_stats(df):
    for jornada, local, visit, link in zip(df.Jornada, df.Local, df.Visitante, df.Link):
        
        dir = f"data/2020/Jornada {jornada}"
        Path(dir).mkdir(parents=True, exist_ok=True)

        if not os.path.exists(f'{dir}/{local} vs {visit}.xlsx'):

            print(jornada, link, local,'vs',visit)

            # Games Stats
            teams_stats = pd.read_html(link.replace('match','match_performance'))

            if len(teams_stats)<2:

                # Create a Pandas Excel writer using XlsxWriter as the engine.
                writer = pd.ExcelWriter(f'{dir}/NA {local} vs {visit}.xlsx', engine='xlsxwriter')

                writer.save()
                writer.close()

                time.sleep(random.randint(5, 10))
                continue

            # Create a Pandas Excel writer using XlsxWriter as the engine.
            writer = pd.ExcelWriter(f'{dir}/{local} vs {visit}.xlsx', engine='xlsxwriter')

            # Splitting Stats by Local - Visit
            local_stats = teams_stats[2].rename(columns={'Unnamed: 0':'Jugador'})
            visit_stats = teams_stats[3].rename(columns={'Unnamed: 0':'Jugador'})

            # Convert the dataframe to an XlsxWriter Excel object.
            local_stats.to_excel(writer, sheet_name=f'{local}', index=False)
            visit_stats.to_excel(writer, sheet_name=f'{visit}', index=False)

            # Close the Pandas Excel writer and output the Excel file.
            writer.save()
            writer.close()

            time.sleep(random.randint(20, 30))
        

In [ ]:
extract_stats(df)